In [1]:
import os
import json
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

C:\ProgramData\anaconda3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 파일 위치 지정
keys_file_path = os.path.join('data', 'api_keys.txt')

# 파일에서 API 키를 로드하는 함수
def load_api_keys(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        keys = json.load(file)
    return keys

# API 키 사용
api_keys = load_api_keys(keys_file_path)
openAI_keys = api_keys['openAI_keys']
pinecone_key1 = api_keys['pinecone_key1']
pinecone_key2 = api_keys['pinecone_key2']

In [3]:
from openai import OpenAI

client = OpenAI(api_key=openAI_keys)

with open('data/documents.txt', 'r', encoding="UTF-8") as f:
    lines = f.readlines()

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# 추후 [:5] 제거시 문서 끝까지
embedding_list = []
for line in tqdm(lines[:5]):
    embedding_list.append(model.encode(line))

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  2.74it/s]


In [4]:
em_vectors = []
for i, em in enumerate(embedding_list):
    em_vectors.append({"id": str(i), "values": em, "metadata": {"assertion": lines[i]}})

In [5]:
from pinecone import Pinecone
pc = Pinecone(api_key=pinecone_key2)

index = pc.Index("mpnet")

batch_size = 100

for i in tqdm(range(0, len(em_vectors), batch_size)):
    batch = em_vectors[i:i+batch_size]
    index.upsert(vectors=batch)

NameError: name 'batch_size' is not defined